In [ ]:
import pickle
from datetime import datetime
from pathlib import Path
from typing import Dict

import evaluate
import numpy as np
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    Trainer,
    TrainingArguments,
)

In [ ]:
DATA_DIR = Path.cwd().parent / "data"
DATASET_PATH = DATA_DIR / "train_anno_2023_08_28_17_05.pkl"

HF_MODEL_NAME = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
IGNORE_ENCODING = -100
RANDOM_SEED = 23

TIMESTAMP = datetime.now().strftime("%Y_%m_%d_%H_%M")
MODEL_DIR = Path.cwd().parent / "models" / f"{HF_MODEL_NAME}_{TIMESTAMP}"

In [ ]:
dataset = pickle.load(DATASET_PATH.open("rb"))

In [ ]:
def split_example_to_max_length(tokens, ner_tags, max_length=300):
    for i in range(0, len(tokens), max_length):
        yield tokens[i : i + max_length], ner_tags[i : i + max_length]

In [ ]:
chunked_dataset = [
    [
        {"tokens": tokens, "ner_tags": ner_tags}
        for note_annotations in doc_annotations
        for tokens, ner_tags in split_example_to_max_length(
            note_annotations["tokens"], note_annotations["ner_tags"]
        )
    ]
    for doc_annotations in dataset
]

In [ ]:
train_dataset, val_dataset = train_test_split(
    chunked_dataset, test_size=0.2, random_state=RANDOM_SEED
)
len(train_dataset), len(val_dataset)

In [ ]:
train_dataset_dict = {
    "tokens": [example["tokens"] for doc in train_dataset for example in doc],
    "ner_tags": [example["ner_tags"] for doc in train_dataset for example in doc],
}

val_dataset_dict = {
    "tokens": [example["tokens"] for doc in val_dataset for example in doc],
    "ner_tags": [example["ner_tags"] for doc in val_dataset for example in doc],
}

In [ ]:
train_dataset_hf = Dataset.from_dict(train_dataset_dict)
val_dataset_hf = Dataset.from_dict(val_dataset_dict)

In [ ]:
labels = sorted({label for sample in train_dataset_hf for label in sample["ner_tags"]})
labels

In [ ]:
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}
id2label, label2id

In [ ]:
def tokenize_and_align_labels(
    examples: Dataset, tokenizer: AutoTokenizer, label2id: Dict
) -> Dataset:
    tokenized_inputs = tokenizer(examples["tokens"], is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(
            batch_index=i
        )  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(IGNORE_ENCODING)
            elif (
                word_idx != previous_word_idx
            ):  # Only label the first token of a given word.
                label_ids.append(label2id[label[word_idx]])
            else:
                label_ids.append(IGNORE_ENCODING)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_NAME)

In [ ]:
tokenized_train_dataset = train_dataset_hf.map(
    lambda examples: tokenize_and_align_labels(examples, tokenizer, label2id),
    batched=True,
)
tokenized_val_dataset = val_dataset_hf.map(
    lambda examples: tokenize_and_align_labels(examples, tokenizer, label2id),
    batched=True,
)

In [ ]:
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["tokens", "ner_tags"])
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["tokens", "ner_tags"])
len(tokenized_train_dataset), len(tokenized_val_dataset)

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    HF_MODEL_NAME, num_labels=len(labels), id2label=id2label, label2id=label2id
)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
seqeval = evaluate.load("seqeval")


def compute_metrics(p):
    dataset_logits, dataset_labels = p
    dataset_preds = np.argmax(dataset_logits, axis=2)

    dataset_filtered_preds = [
        [
            labels[token_pred]
            for (token_pred, token_label) in zip(doc_preds, doc_labels)
            if token_label != IGNORE_ENCODING
        ]
        for doc_preds, doc_labels in zip(dataset_preds, dataset_labels)
    ]
    dataset_filtered_labels = [
        [
            labels[token_label]
            for (token_pred, token_label) in zip(doc_preds, doc_labels)
            if token_label != IGNORE_ENCODING
        ]
        for doc_preds, doc_labels in zip(dataset_preds, dataset_labels)
    ]

    results = seqeval.compute(
        predictions=dataset_filtered_preds, references=dataset_filtered_labels
    )
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
training_args = TrainingArguments(
    output_dir=MODEL_DIR,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()